In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfin
else:
    try:
        import ufl
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl
        import dolfin

In [ ]:
import numpy as np
from petsc4py import PETSc
assert not np.issubdtype(PETSc.ScalarType, np.complexfloating)

In [ ]:
mesh = dolfin.UnitIntervalMesh(3)

In [ ]:
V = dolfin.FunctionSpace(mesh, "CG", 1)
assert V.dim() == 4

In [ ]:
u = dolfin.TrialFunction(V)
v = dolfin.TestFunction(V)

In [ ]:
dx = ufl.dx

In [ ]:
f = dolfin.Function(V)
f.vector()[:] = np.arange(1, V.dim() + 1)

In [ ]:
a = u * v * dx
F = f * v * dx

In [ ]:
A = dolfin.assemble(a)
b = dolfin.assemble(F)

In [ ]:
A = dolfin.as_backend_type(A)
b = dolfin.as_backend_type(b)

In [ ]:
solution = dolfin.Function(V)

In [ ]:
dolfin.solve(A, solution.vector(), b)

In [ ]:
assert np.allclose(solution.vector().vec().getArray(), np.arange(1, V.dim() + 1))

In [ ]:
ksp = PETSc.KSP().create()
ksp.setOperators(A.mat())
ksp.solve(b.vec(), solution.vector().vec())
assert np.allclose(solution.vector().vec().getArray(), np.arange(1, V.dim() + 1))

In [ ]:
for package in ("mumps", "superlu", "superlu_dist"):
    ksp = PETSc.KSP().create()
    ksp.setOperators(A.mat())
    ksp.setType("preonly")
    pc = ksp.getPC()
    pc.setType("lu")
    pc.setFactorSolverType(package)
    ksp.solve(b.vec(), solution.vector().vec())
    assert np.allclose(solution.vector().vec().getArray(), np.arange(1, V.dim() + 1))

In [ ]:
grad = ufl.grad
inner = ufl.inner

In [ ]:
k = inner(grad(u), grad(v)) * dx

In [ ]:
K = dolfin.assemble(k)
K = dolfin.as_backend_type(K)

In [ ]:
expected = (0, 10.8, 54, 108)

In [ ]:
eigensolver = dolfin.SLEPcEigenSolver(K, A)
eigensolver.parameters["problem_type"] = "gen_non_hermitian"
eigensolver.parameters["spectrum"] = "smallest real"
eigensolver.solve(V.dim())
assert eigensolver.get_number_converged() == len(expected)
for (i, eig_i_ex) in enumerate(expected):
    eig_i_r, eig_i_i = eigensolver.get_eigenvalue(i)
    assert np.isclose(eig_i_r, eig_i_ex)
    assert np.isclose(eig_i_i, 0)

In [ ]:
from slepc4py import SLEPc

In [ ]:
eps = SLEPc.EPS().create()
eps.setOperators(K.mat(), A.mat())
eps.setProblemType(SLEPc.EPS.ProblemType.GHEP)
eps.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)
eps.solve()
assert eps.getConverged() == len(expected)
for (i, eig_i_ex) in enumerate(expected):
    eig_i = eps.getEigenvalue(i)
    assert np.isclose(eig_i.real, eig_i_ex)
    assert np.isclose(eig_i.imag, 0)

In [ ]:
for package in ("mumps", "superlu", "superlu_dist"):
    eps = SLEPc.EPS().create()
    eps.setOperators(K.mat(), A.mat())
    eps.setProblemType(SLEPc.EPS.ProblemType.GHEP)
    eps.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_REAL)
    eps.setTarget(1)
    st = eps.getST()
    st.setType(SLEPc.ST.Type.SINVERT)
    st.setShift(1)
    ksp = st.getKSP()
    ksp.setType("preonly")
    pc = ksp.getPC()
    pc.setType("lu")
    pc.setFactorSolverType(package)
    eps.solve()
    assert eps.getConverged() == len(expected)
    for (i, eig_i_ex) in enumerate(expected):
        eig_i = eps.getEigenvalue(i)
        assert np.isclose(eig_i.real, eig_i_ex)
        assert np.isclose(eig_i.imag, 0)

In [ ]:
%%bash

export LD_PRELOAD=""
ERROR_LIBRARIES=($(find /root/.cache/dijitso -name '*\.so' -exec \
    bash -c 'ldd $0 | grep libstdc++.so.6 1>/dev/null 2>/dev/null && echo $0' {} \;))
if [ ${#ERROR_LIBRARIES[@]} -eq 0 ]; then
    echo "No reference to libstdc++.so was found"
else
    for ERROR_LIBRARY in "${ERROR_LIBRARIES[@]}"; do
        echo "Error: library $ERROR_LIBRARY depends on libstdc++.so"
        ldd -v $ERROR_LIBRARY
    done
    false
fi